*Import Dependencies*

In [2]:
import os
import sys
import warnings
import pandas 
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

*Load Dataset*

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

*EDA*

In [4]:
df.drop("link", axis=1, inplace = True)


In [5]:
df.head()

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [6]:
df = df[0:20000]
df.drop_duplicates(subset="song", inplace=True)
df.reset_index(drop = True, inplace = True)

In [7]:
df.rename(columns = {"artist":'Artist','song':'Song','text':'Lyrics'},inplace = True)

In [8]:
df.head(5)

,Artist,Song,Lyrics
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [9]:
df.isnull().sum()

Artist    0
Song      0
Lyrics    0
dtype: int64

In [10]:
df.duplicated().sum()

0

In [11]:
df.columns

Index(['Artist', 'Song', 'Lyrics'], dtype='object')

In [12]:
df['Combined_features'] = df['Artist']+" "+df['Song']+" "+df['Lyrics']

*Function for cleaning data*

In [13]:
import re
import nltk
from nltk.corpus import stopwords
def cleaning(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

In [14]:
df['Combined_features'] = df['Combined_features'].apply(func = cleaning)

*Vectorization*

In [15]:
tfv = TfidfVectorizer(max_features = 20000)

*Data Transformation*

In [16]:
tfv_matrix = tfv.fit_transform(df["Combined_features"])

*Cosine Similarities Matrix*

In [20]:
cosine_sim = cosine_similarity(tfv_matrix)

In [18]:
song_user_likes = "Hope"
song_index = df[df.Song == song_user_likes].index[0]

In [21]:
similar_songs = list(enumerate(cosine_sim[song_index]))

In [22]:
similar_song_sorted = sorted(similar_songs,key = lambda x:x[1],reverse = True)

*Recommender System Creation*

In [23]:
def recommended_songs(song_user_likes):
    song_index = df[df.Song == song_user_likes].index[0]
    similar_songs = list(enumerate(cosine_sim[song_index]))
    similar_song_sorted = sorted(similar_songs, key=lambda x:x[1], reverse=True)
    
    for song in similar_song_sorted[1:11]:
        similar_songs = df[df.index == song[0]]["Song"].values[0]
        print(similar_songs)

In [24]:
recommended_songs("Cool")

Boy Boy (I'm The Boy)
Hot Boy
Only A Boy
Boy
Silly Boy
Bad Boy
Selfish Girl
The House Is Rockin'
Sk8er Boi
Dear Boy


In [25]:
import joblib

# Assuming `tfv` is your trained TfidfVectorizer and `cosine_sim` is your cosine similarity matrix
joblib.dump(tfv, 'tfidf_vectorizer.pkl')
joblib.dump(cosine_sim, 'cosine_similarity_matrix.pkl')

print("Model and cosine similarity matrix saved.")


Model and cosine similarity matrix saved.
